![](image.jpg)

In 1986, a group of urologists in London published a research paper in The British Medical Journal that compared the effectiveness of two different methods to remove kidney stones. Treatment A was open surgery (invasive), and treatment B was percutaneous nephrolithotomy (less invasive). When they looked at the results from 700 patients, treatment B had a higher success rate. However, when they only looked at the subgroup of patients different kidney stone sizes, treatment A had a better success rate. What is going on here? This known statistical phenomenon is called Simpon’s paradox. Simpon's paradox occurs when trends appear in subgroups but disappear or reverse when subgroups are combined.

## The Data
Available on `kidney_stone_data.csv`

| Column      | Type      | Description |
|-------------|-----------|-------------|
| `treatment` | discrete  | Treatment method, indicated by A or B |
| `stone_size`| discrete  | Size of the kidney stone, categorized as 'small' or 'large' |
| `success`   | discrete  | Outcome of the treatment: 1=successful, 0=unsuccessful |

In this project, we are going to explore Simpon’s paradox using multiple regression and other statistical tools. 
Our main goal is to determine if Treatment A is superior to Treatment B after accounting for the severity of the kidney stones. 

In [2]:
# Load the necessary packages
library(readr)
library(dplyr)
library(ggplot2)
library(broom)

# Load the data
data <- read_csv("kidney_stone_data.csv")

# Inspect the first five rows
head(data, 5)


# Calculate the number and frequency of success and failure of each treatment
data %>% 
  group_by(treatment, success) %>%
  summarise(N = n()) %>%
  mutate(Freq = round(N/sum(N),3))

# Calculate number and frequency of success and failure by stone size for each treatment
sum_data <- 
  data %>% 
  group_by(treatment, stone_size, success) %>%
  summarise(N = n()) %>%
  mutate(Freq = round(N/sum(N),3))

# Print out the data frame we just created
sum_data

# Run a Chi-squared test
trt_ss <- chisq.test(data$treatment, data$stone_size)

# Print out the result in tidy format 
tidy(trt_ss)

# Run a multiple logistic regression
logistic_model <- glm(data = data, success ~ stone_size + treatment, family = 'binomial')

# Print out model coefficient table in tidy format
tidy(logistic_model)

# Save the tidy model output into an object
tidy_m <- tidy(logistic_model)

# Is small stone more likely to be a success after controlling for treatment option effect?
small_high_success <- "Yes"

# Is treatment A significantly better than B?
A_B_sig <- "No"

Rows: 700 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): treatment, stone_size
dbl (1): success

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


treatment,stone_size,success
<chr>,<chr>,<dbl>
B,large,1
A,large,1
A,large,0
A,large,1
A,large,1


`summarise()` has grouped output by 'treatment'. You can override using the
`.groups` argument.
New names:
• `` -> `...1`


treatment,success,N,Freq
<chr>,<dbl>,<int>,<dbl>
A,0,77,0.220
A,1,273,0.780
B,0,61,0.174
B,1,289,0.826


`summarise()` has grouped output by 'treatment', 'stone_size'. You can override
using the `.groups` argument.
New names:
• `` -> `...1`


treatment,stone_size,success,N,Freq
<chr>,<chr>,<dbl>,<int>,<dbl>
A,large,0,71,0.270
A,large,1,192,0.730
A,small,0,6,0.069
A,small,1,81,0.931
B,large,0,25,0.312
B,large,1,55,0.688
B,small,0,36,0.133
B,small,1,234,0.867


statistic,p.value,parameter,method
<dbl>,<dbl>,<int>,<chr>
189.3557,4.395009e-43,1,Pearson's Chi-squared test with Yates' continuity correction


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.0332140,0.1344695,7.683629,1.546436e-14
stone_sizesmall,1.2605654,0.2390027,5.274272,1.332838e-07
treatmentB,-0.3572287,0.2290792,-1.559411,1.188991e-01
